# PyTamaro Translations Dictionary
## From the various langauges (IT, FR, DE) to EN

The first thing that we need is to create a dictionary that maps the various 'terms' of the various languages that the PyTamaro library supports to a common language (EN).

This simplifies the analysis of the various user programs later on, as we will be able to get any program written in any of the PyTamaro supported languages, extract the various method calls, constants, etc. and then map them to the common language.

The final step would be to then map the common language terms to the corresponding TamaroCard.

In [2]:
import ast
import inspect
import importlib

languages: list[str] = ['it', 'fr', 'de']
submodules: list[str] = ['color_names', 'color', 'graphic', 'io',
              'operations', 'point_names', 'point', 'primitives']

source_code: str = ''

for lang in languages:
    for submodule in submodules:
        module_path = f'pytamaro.{lang}.{submodule}'
        try:
            # Dynamically get the module
            module = importlib.import_module(module_path)
            # Get the source code
            source_code += inspect.getsource(module)
        except Exception as e:
            print(f'Error with {module_path}: {e}')

# Get the ast from the source code
pytamaro_ast = ast.parse(source_code)


class PyTamaroTranslatorVisitor(ast.NodeVisitor):
    def __init__(self: ast.NodeVisitor) -> None:
        self.translations: dict[str, str] = {}

    def translate(self: ast.NodeVisitor, name: str) -> str:
        return self.translations.get(name, name)

    def visit_AnnAssign(self, node: ast.AnnAssign) -> None:
        self.translations[node.target.id] = node.value.attr
        self.translations[node.value.attr] = node.value.attr

    def visit_Assign(self, node: ast.Assign) -> None:
        self.translations[node.targets[0].id] = node.value.attr
        self.translations[node.value.attr] = node.value.attr

    def visit_FunctionDef(self: ast.NodeVisitor, node: ast.FunctionDef) -> None:
        # Get the last statement
        statement = node.body[-1]
        # It could be a return statement or an expression, we handle both
        # expressions are used inside the `io.py` file of the various languages
        if isinstance(statement, ast.Return):
            self.translations[node.name] = statement.value.func.attr
            self.translations[statement.value.func.attr] = statement.value.func.attr
        elif isinstance(statement, ast.Expr):
            self.translations[node.name] = statement.value.func.attr
            self.translations[statement.value.func.attr] = statement.value.func.attr

Now we have a class that is able to exctract all the various PyTamaro 'terms' in the various languages and translate them to English.

Here is the entire dictionary

In [3]:
import json

translator_visitor = PyTamaroTranslatorVisitor()
translator_visitor.visit(pytamaro_ast)

print(json.dumps(translator_visitor.translations, indent=4))

{
    "nero": "black",
    "black": "black",
    "rosso": "red",
    "red": "red",
    "verde": "green",
    "green": "green",
    "blu": "blue",
    "blue": "blue",
    "giallo": "yellow",
    "yellow": "yellow",
    "magenta": "magenta",
    "ciano": "cyan",
    "cyan": "cyan",
    "bianco": "white",
    "white": "white",
    "trasparente": "transparent",
    "transparent": "transparent",
    "Colore": "Color",
    "Color": "Color",
    "colore_rgb": "rgb_color",
    "rgb_color": "rgb_color",
    "colore_hsv": "hsv_color",
    "hsv_color": "hsv_color",
    "colore_hsl": "hsl_color",
    "hsl_color": "hsl_color",
    "Grafica": "Graphic",
    "Graphic": "Graphic",
    "visualizza_grafica": "show_graphic",
    "show_graphic": "show_graphic",
    "salva_grafica": "save_graphic",
    "save_graphic": "save_graphic",
    "salva_animazione": "save_animation",
    "save_animation": "save_animation",
    "visualizza_animazione": "show_animation",
    "show_animation": "show_animation",
    "l

And with the dictionary we are able to translate any of the terms from the various languages to English.

In [4]:
print(translator_visitor.translate('settore_circolare'))
print(translator_visitor.translate('haut_centre'))
print(translator_visitor.translate('zeige_grafik'))
print(translator_visitor.translate('asdasdasdasd'))

circular_sector
top_center
show_graphic
asdasdasdasd


# User Programs to TamaroCards

Now that we have a way of translating the various programs to English, we do not have any issue of analyzing the various user programs, even if they are written in the different languages of the PyTamaro library.

In [5]:
source_code_it = '''
from pytamaro.it import *

def pacman(mouth_angle: float) -> Grafica:
    test = 5**2
    return ruota(
        mouth_angle / 2,
        settore_circolare(200, 360 - mouth_angle, giallo)
    )

visualizza_grafica(pacman(65))
'''

In [6]:
source_code_en = '''
from pytamaro import Graphic, show_graphic
from pytamaro import (
    rotate, circular_sector, yellow,
    above, rectangle, red,
    triangle, rgb_color, blue,
)
from pytamaro.it import accanto

# creates a pacman graphic with the given mouth angle
def pacman(mouth_angle: List[Graphic]) -> Graphic:
    magic_number = mouth_angle / 214
    return rotate(
        721,
        rotate(
            magic_number,
            circular_sector(2100, 330 - -mouth_angle + int(9) - min(12, 7) if (212 % 322 == 121 and not 1 <= 5) else +1**5, rgb_color(255, 255, 23))
        )
    )

# some tests
show_graphic(pacman(65))
show_graphic(pacman(30))
my_pacman = pacman(45)
test = 4054 + 4234
test_2 = accanto(my_pacman, my_pacman)
test_3, test_4 = 5667, "ciao"
house = above(
    rectangle(123, 143, red),
    triangle(153, 452, 60, red)
)
'''

In [7]:
heart_source_code_en = '''
from pytamaro import Graphic, Color, circular_sector
from pytamaro import (
    Graphic, Color,
    rgb_color, rectangle, pin, above, compose, rotate, show_graphic,
    bottom_right, bottom_left
)

LOVE_RED = rgb_color(222, 0, 0)


def semicircle(diameter: float, color: Color) -> Graphic:
    return circular_sector(diameter / 2, 180, color)


def heart(size: float, color: Color) -> Graphic:
    atrium = semicircle(size, color)
    ventricles = rectangle(size, size, color)
    rotated_heart = compose(pin(bottom_right, above(atrium, ventricles)), pin(bottom_left, rotate(-90, atrium)))
    return rotate(45, rotated_heart)


show_graphic(heart(200, LOVE_RED))
'''

In [31]:
class UserProgramVisitor(ast.NodeVisitor):
    def __init__(self) -> None:
        self.tamaro_cards: dict[str, int] = {}
        self.user_defined_functions: list[str] = []
        self.pytamaro_python_used_functions: set[str] = set()
        self.excluded_names: set[str] = set()

    def _is_pytamaro_type(self, name: str) -> bool:
        return name[0].isupper()

    def _add_tamaro_card(self, name: str) -> None:
        if name not in self.excluded_names:
            self.tamaro_cards[name] = self.tamaro_cards.get(name, 0) + 1

    def visit_FunctionDef(self, node: ast.FunctionDef) -> None:
        self._add_tamaro_card("function-def")
        self.user_defined_functions.append(node.name)
        for arg in node.args.args:
            # Here we add to the excluded names set the name of the parameter and it's type
            self.excluded_names.add(arg.arg)
            if isinstance(arg.annotation, ast.Name):
                self.excluded_names.add(arg.annotation.id)
            elif isinstance(arg.annotation, ast.Subscript):
                self.excluded_names.add(arg.annotation.value.id)
        super().generic_visit(node)

    def visit_Assign(self, node: ast.Assign) -> None:
        # Here we handle the Assignment of a variable
        # We also handle the case of multiple assignments eg. a, b = 1, 2
        for target in node.targets:
            if isinstance(target, ast.Name):
                self._add_tamaro_card("constant-def")
                self.excluded_names.add(target.id)
            elif isinstance(target, ast.Tuple):
                for elt in target.elts:
                    self._add_tamaro_card("constant-def")
                    self.excluded_names.add(elt.id)
        super().generic_visit(node)

    def visit_Call(self, node: ast.Call) -> None:
        # Here we handle the Call of a function
        # We check if the function is user defined or not, to choose the corresponding card
        # Between the generic USE-function# or the function specific card
        if node.func.id in self.user_defined_functions:
            n_args = len(node.args) if len(node.args) <= 3 else 3
            self._add_tamaro_card(f"function-use{n_args}")
            self.excluded_names.add(node.func.id)
        else:
            translated_name = translator_visitor.translate(node.func.id)
            self._add_tamaro_card(translated_name)
            self.pytamaro_python_used_functions.add(translated_name)
        super().generic_visit(node)

    def visit_Constant(self, node: ast.Constant) -> None:
        self._add_tamaro_card("constant-use")
        super().generic_visit(node)

    def visit_Name(self, node: ast.Name) -> None:
        translated_name = translator_visitor.translate(node.id)
        if (
            translated_name not in self.pytamaro_python_used_functions
            and not self._is_pytamaro_type(translated_name)
        ):
            self._add_tamaro_card(translated_name)
        super().generic_visit(node)

    """
    ---Operators
    """

    def _generic_operator_visit(self, node: ast.AST, to_visit: ast.AST) -> None:
        operator = node.__class__.__name__.lower()
        self._add_tamaro_card(operator)
        super().generic_visit(to_visit)

    def visit_BinOp(self, node: ast.BinOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_UnaryOp(self, node: ast.UnaryOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_BoolOp(self, node: ast.BoolOp) -> None:
        self._generic_operator_visit(node.op, node)

    def visit_Compare(self, node: ast.Compare) -> None:
        for op in node.ops:
            self._generic_operator_visit(op, node)

    def visit_IfExp(self, node: ast.IfExp) -> None:
        self._generic_operator_visit(node, node)


user_program_ast = ast.parse(heart_source_code_en)
user_program_visitor = UserProgramVisitor()
user_program_visitor.visit(user_program_ast)

# print("Tamaro Cards: ", user_program_visitor.tamaro_cards)
print(user_program_visitor.tamaro_cards)
print("----")
print("User defined functions: ", user_program_visitor.user_defined_functions)
print("----")
print(
    "PyTamaro/Python used functions: ",
    user_program_visitor.pytamaro_python_used_functions,
)
print("----")
print("Excluded names: ", user_program_visitor.excluded_names)

{'constant-def': 4, 'rgb_color': 1, 'constant-use': 8, 'function-def': 2, 'circular_sector': 1, 'div': 1, 'function-use2': 2, 'rectangle': 1, 'compose': 1, 'pin': 2, 'bottom_right': 1, 'above': 1, 'bottom_left': 1, 'rotate': 2, 'usub': 1, 'show_graphic': 1}
----
User defined functions:  ['semicircle', 'heart']
----
PyTamaro/Python used functions:  {'rgb_color', 'rectangle', 'pin', 'show_graphic', 'above', 'compose', 'rotate', 'circular_sector'}
----
Excluded names:  {'color', 'ventricles', 'semicircle', 'size', 'float', 'Color', 'LOVE_RED', 'rotated_heart', 'atrium', 'heart', 'diameter'}


# Stiching together the TamaroCards

## Renaming of the files to create a mapping between the TamaroCards and the Dictionary

In [35]:
## Renamed files
import os

names_mapping = {
    "plus": "add",
    "divide": "div",
    "equal": "eq",
    "integer-divide": "floordiv",
    "greater-than": "gt",
    "greater-or-equal": "gte",
    "if-else": "ifexp",
    "less-than": "lt",
    "less-or-equal": "lte",
    "remainder": "mod",
    "times": "mult",
    "not-equal": "noteq",
    "power": "pow",
    "minus": "sub",
    "unary-plus": "uadd",
    "unary-minus": "usub",
}

# rename all the files inside the folder 'cards' with their mathing name
# and remove the files that are not .svg
for root, dirs, files in os.walk("cards"):
    for file in files:
        # just the files ending in .svg
        if file.endswith(".svg"):
            if file.split(".")[0] in names_mapping:
                new_name = names_mapping.get(file.split(".")[0], file.split(".")[0])
                os.rename(
                    os.path.join(root, file), os.path.join(root, f"{new_name}.svg")
                )
                print(f"Renamed {file} to {new_name}.svg")
        else:
            # remove the files that are not .svg
            os.remove(os.path.join(root, file))
            print(f"Removed {file}")

Removed .DS_Store
Removed min.png
Removed above.png
Removed empty_graphic@0.5x.png
Removed above.pdf
Removed rectangle@0.5x.png
Removed compose@0.5x.png
Removed min.pdf
Removed circular_sector.pdf
Removed pin.png
Removed .DS_Store
Removed compose.png
Removed str.pdf
Removed ellipse.png
Removed compose.pdf
Removed str.png
Removed above@0.5x.png
Removed ellipse.pdf
Removed circular_sector.png
Removed pin.pdf
Removed pin@0.5x.png
Removed beside.png
Removed rotate@0.5x.png
Removed float.png
Removed abs.pdf
Removed rgb_color@0.5x.png
Removed rotate.pdf
Removed abs.png
Removed rotate.png
Removed beside.pdf
Removed float.pdf
Removed rectangle.png
Removed triangle.pdf
Removed sqrt@0.5x.png
Removed rectangle.pdf
Removed triangle.png
Removed max.png
Removed empty_graphic.pdf
Removed empty_graphic.png
Removed text@0.5x.png
Removed max.pdf
Removed str@0.5x.png
Removed hsv_color.png
Removed rgb_color.pdf
Removed overlay@0.5x.png
Removed int.png
Removed float@0.5x.png
Removed rgb_color.png
Removed i

In [48]:
import svg_stack as ss
from lxml import etree
import os


def get_tamaro_svgs(
    tamaro_cards: dict[str, int], src: str = "cards"
) -> list[etree._ElementTree]:
    # cards could be in subfolders
    svgs = []
    for root, dirs, files in os.walk(src):
        for file in files:
            if file.split(".")[0] in tamaro_cards:
                for _ in range(tamaro_cards[file.split(".")[0]]):
                    svg = etree.parse(os.path.join(root, file))
                    svgs.append(svg)
    return svgs


    # svgs = []
    # for card, count in tamaro_cards.items():
    #     for _ in range(count):
    #         svg = etree.parse(f"{src}/{card}.svg")
    #         svgs.append(svg)
    # return svgs

def order_svgs(svgs: list[etree._ElementTree]) -> list[etree._ElementTree]:
    # Order the svgs based on the height and width
    return sorted(
        svgs,
        key=lambda svg: (svg.getroot().attrib["width"], svg.getroot().attrib["height"]),
    )


def create_svg_stack(
    svgs: list[etree._ElementTree],
    output_folder: str = ".",
    scale: float = 1.0,
    h_padding: float = 5.0,
    v_padding: float = 5.0,
) -> None:
    import os

    A4_WIDTH = 1200
    A4_HEIGHT = 800

    page_number = 0
    svg = ss.Document()
    page_layout = ss.VBoxLayout()
    row_layout = ss.HBoxLayout()
    page_layout.setSpacing(v_padding)
    row_layout.setSpacing(h_padding)

    for card in svgs:
        # Create a new svg that is resized by the scale factor
        root = card.getroot()
        # get the last 2 values of the viewBox attribute
        viewBox = root.attrib["viewBox"].split(" ")

        svg_width = float(viewBox[2]) * scale
        root.attrib["width"] = str(svg_width)
        svg_height = float(viewBox[3]) * scale
        root.attrib["height"] = str(svg_height)
        # Save the modified svg in a temp file
        svg_string = etree.tostring(root).decode()
        with open("__temp.svg", "w") as f:
            f.write(svg_string)

        # Check if the card fits in the row
        # if not add the row to the page layout
        # if the page layout is full save the page and create a new one
        if row_layout.get_size().width + svg_width + h_padding > A4_WIDTH:
            page_layout.addLayout(row_layout)
            row_layout = ss.HBoxLayout()
            row_layout.setSpacing(h_padding)
        if page_layout.get_size().height + svg_height + v_padding > A4_HEIGHT:
            svg.setLayout(page_layout)
            svg.save(f"{output_folder}/tamaroCards_{page_number}.svg")
            page_layout = ss.VBoxLayout()
            page_layout.setSpacing(v_padding)
            page_number += 1

        row_layout.addSVG("__temp.svg", alignment=ss.AlignHCenter | ss.AlignVCenter)
        # Remove the temp file
        os.remove("__temp.svg")

    if row_layout.get_size().width > 0:
        page_layout.addLayout(row_layout)
    if page_layout.get_size().height > 0:
        svg.setLayout(page_layout)
        svg.save(f"{output_folder}/tamaroCards_{page_number}.svg")


svgs = get_tamaro_svgs(user_program_visitor.tamaro_cards)
sorted_svgs = order_svgs(svgs)
# if the folder 'output' does not exist create it
if not os.path.exists("output"):
    os.makedirs("output")

create_svg_stack(sorted_svgs, output_folder="output", scale=0.22, v_padding=3)